<p style="font-family:Calibri; font-size:1.75em; color:white; padding:5px 5px; text-align: left; height:50px; border-bottom: 1px solid orange; background:#f5eab8"></p>

<p style="font-size:2.2em; color:white; padding:25px; border-bottom: 5px solid #ffb701; background:#143157;"> 
    Sales Target by Supplier - <span style="font-size:.75em; color: #11a1f5;"> from excel file </span></p>
Read data from csv in Pandas

In [76]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', 85) # set Jupeter note book to dispay 85 columns

<p style="padding:5px; border-bottom:3px solid orange; background:#3c5c75; margin:25px 0 -7px 0;"> 
   <span style="font-size:1.5em; color:#f7ca25;" >Step 1 -</span>
   <span style="font-size:1.5em; color:white;">Read CSV files  - </span> 
   <span style="font-size:1.2em; color:#17cffc;">"Employee Target"</span>
</p> 

In [77]:
df = pd.read_csv("Data/2023/I1_Hrm/Employees/empTarget.csv", index_col=False ,encoding='unicode_escape').fillna(0)

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 1a - Remove Duplicate Line and Deleted Record</p>

In [78]:
DataA = df.drop_duplicates(subset=['empCode'], keep='last')
DataB = DataA.drop(DataA.loc[DataA['comment']=='Delete'].index)

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 1b - Combine Employee Code and Transaction Time</p>

In [79]:
DataB['combined'] = DataB['empCode'].astype(str)+" "+DataB['comment']

TypeError: can only concatenate str (not "float") to str

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 1c- Select the Combine Column and convert it into a "LIST"</p>

In [ ]:
List = DataB["combined"] #.tolist()

<p style="padding:5px; border-bottom:3px solid orange; background:#3c5c75; margin:25px 0 -7px 0;"> 
   <span style="font-size:1.5em; color:#f7ca25;" >Step 2 -</span>
   <span style="font-size:1.5em; color:white;">Read CSV files  - </span> 
   <span style="font-size:1.2em; color:#17cffc;">" Employee Target Detail"</span>
</p> 

In [19]:
df2 = pd.read_csv("Data/2023/I1_Hrm/Employees/empTargetDetail.csv", index_col=False ,encoding='unicode_escape').fillna(0)
### Combine Employee Code and Transaction Time.................
df2['combined'] = df2['empCode'].astype(str)+" "+df2['traDate']

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 3a- Filtere the 'Employee Target Detail' by Employee "Target List"</p>

In [20]:
filtered_df = df2[df2['combined'].isin(List)]

In [21]:
### Convert 'salesTarget' astype to "FLOAT" to avoid error if any numaric value contain float type.
filtered_df = filtered_df.astype({'salesTarget':'float'})

<p style="padding:5px; border-bottom:3px solid orange; background:#3c5c75; margin:25px 0 -7px 0;"> 
   <span style="font-size:1.5em; color:#f7ca25;" >Step 4 -</span>
   <span style="font-size:1.5em; color:white;">Create Pivot Table - </span> 
   <span style="font-size:1.2em; color:#17cffc;"> by Supplier </span>
</p> 

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 4a- Read Item Master Data CSV file Selected Column and convert into List</p>

In [22]:
req_cols = ['itmCode','supplier']
Supp = pd.read_csv("Data/H1_Items/Item/ItemMaster.csv", usecols=req_cols, encoding='unicode_escape').fillna(0)
supList = Supp.values.tolist() ##Convert into List

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 4b- Map List with Item Code and get Supplier Name</p>

In [23]:
supDict = {code: name for code, name in supList}
filtered_df['supplier'] = filtered_df['itemCode'].map(supDict)

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 4b- Create Pivot Table by Supplier</p>

In [24]:
### PIVOT BY Supplier------------------------------------
suppPT = pd.pivot_table(filtered_df,index=["supplier","itemDesc"],values=["salesTarget","qty"],aggfunc=np.sum)
suppPT= suppPT.reset_index()
suppGBy = suppPT.groupby(['supplier']).agg({'salesTarget': 'sum', 'qty': 'sum'}) 
suppGBy.loc['z. Grand Total'] = suppGBy.sum()
suppGBy.reset_index(inplace=True)
suppConcat = pd.concat([suppPT, suppGBy])
suppSort = suppConcat.sort_values(['supplier', 'itemDesc'], ascending = [True, True], na_position ='last')
suppSort['itemDesc'] = suppSort['itemDesc'].fillna('Total')

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 4c- Supplier Traget Percentager by ItemTotal</p>

In [25]:
### Convert into Dictionary--------------------
suppGBy = suppGBy[['supplier','salesTarget']]
t2List = suppGBy.values.tolist() ##Convert into List
mapDic2 = {code: name for code, name in t2List}
suppSort['Total'] = suppSort['supplier'].map(mapDic2)
suppSort['Percentage'] = (suppSort['salesTarget'] / suppSort['Total'])*100

In [26]:
### to Hide and Show----------------------------
suppSort['Department'] = suppSort['supplier']
suppSort['Vertical'] = suppSort['supplier']
suppSort.loc[suppSort["itemDesc"] != "Total", ["Vertical"]] = '---' #  rlSort["status"]

In [27]:
supData=[]
for i in range(suppSort.shape[0]):
        temp=suppSort.iloc[i]
        supData.append(dict(temp))

In [75]:
supData

NameError: name 'supData' is not defined

<p style="font-family:Calibri; font-size:1.75em; color:white; padding:5px 5px; text-align: left; height:50px; border-bottom: 1px solid orange; background:#f5eab8"></p>

<p style="font-size:2.2em; color:white; padding:25px; border-bottom: 5px solid #ffb701; background:#143157;"> 
    Sales Target by Employee - <span style="font-size:.75em; color: #11a1f5;"> from excel file </span></p>
Read data from csv in Pandas

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 1 - </span>
   <span style="font-size:1.4em; color:#e39607;">Get Company and Year - </span> 
   <span style="font-size:1.2em; color:#209ebd;">to Load Data </span>
</p> 

In [80]:
import pandas as pd
import numpy as np
dfStd = pd.read_csv('data/basInfo.csv')
Year = str(dfStd.loc[0, 'year'])
coFolder = str(dfStd.loc[0, 'coFolder'])

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 2 - </span>
   <span style="font-size:1.4em; color:#e39607;">Read Employee- </span> 
   <span style="font-size:1.2em; color:#209ebd;">salesTarget.csv File </span>
</p> 

In [81]:
fPath = "Data/" +coFolder+ "/" +Year+ "/I1_Hrm/Targets/salesTarget.csv"
df = pd.read_csv(fPath, index_col=False, encoding='unicode_escape').fillna('')

In [82]:
df['unique'] = df['empCode'].astype(str)+" "+df['sNo'].astype(str) ### create a unique column

In [83]:
DataA = df.drop_duplicates(subset=['unique'], keep='last')  ### drop duplicate value
DataB = DataA.drop(DataA.loc[DataA['action']=='Deleted'].index) ### drop deleted record

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 3 - </span>
   <span style="font-size:1.4em; color:#e39607;">Create Pivot Table - </span> 
   <span style="font-size:1.2em; color:#209ebd;">sby Employee </span>
</p> 

In [84]:
empPT = pd.pivot_table(DataB,index=["empCName","itemDesc"],values=["salesTarget","qty"],aggfunc=np.sum)
empPT = empPT.reset_index()
empGBy = empPT.groupby(['empCName']).agg({'salesTarget': 'sum', 'qty': 'sum'}) 
empGBy.loc['z. Grand Total'] = empGBy.sum()
empGBy.reset_index(inplace=True)
empConcat = pd.concat([empPT, empGBy])
empSort = empConcat.sort_values(['empCName', 'itemDesc'], ascending = [True, True], na_position ='last')
empSort['itemDesc'] = empSort['itemDesc'].fillna('Total')

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 4 - </span>
   <span style="font-size:1.4em; color:#e39607;">Employee Traget Percentager by Item</span> 
</p> 

In [85]:
### Convert into Dictionary--------------------
empGBy = empGBy[['empCName','salesTarget']]
t1List = empGBy.values.tolist() ##Convert into List
mapDic1 = {code: name for code, name in t1List}
empSort['Total'] = empSort['empCName'].map(mapDic1)
empSort['Percentage'] = (empSort['salesTarget'] / empSort['Total'])*100

In [87]:
### to Hide and Show----------------------------
empSort['Department'] = empSort['empCName']
empSort['Vertical'] = empSort['empCName']
empSort.loc[empSort["itemDesc"] != "Total", ["Vertical"]] = '---' #  rlSort["status"]

In [88]:
empData=[]
for i in range(empSort.shape[0]):
        temp=empSort.iloc[i]
        empData.append(dict(temp))

In [89]:
empData

[{'empCName': 'Allen Joseph Anora',
  'itemDesc': 'Bruun Mci ',
  'qty': 6,
  'salesTarget': 520800,
  'Total': 1500275,
  'Percentage': 34.71363583343054,
  'Department': 'Allen Joseph Anora',
  'Vertical': '---'},
 {'empCName': 'Allen Joseph Anora',
  'itemDesc': 'Microneedling RF ',
  'qty': 10,
  'salesTarget': 314650,
  'Total': 1500275,
  'Percentage': 20.972821649364285,
  'Department': 'Allen Joseph Anora',
  'Vertical': '---'},
 {'empCName': 'Allen Joseph Anora',
  'itemDesc': 'Microneedling RF - HandPiece ',
  'qty': 41,
  'salesTarget': 193725,
  'Total': 1500275,
  'Percentage': 12.912632684007933,
  'Department': 'Allen Joseph Anora',
  'Vertical': '---'},
 {'empCName': 'Allen Joseph Anora',
  'itemDesc': 'SVTA Radio ',
  'qty': 8,
  'salesTarget': 201600,
  'Total': 1500275,
  'Percentage': 13.43753645165053,
  'Department': 'Allen Joseph Anora',
  'Vertical': '---'},
 {'empCName': 'Allen Joseph Anora',
  'itemDesc': 'SVTA Radio Hand Pice 1 ',
  'qty': 15,
  'salesTarget'

<p style="font-size:2.2em; color:white; padding:25px; border-bottom: 5px solid #ffb701; background:#143157;"> 
    Sales Target by Employee - <span style="font-size:.75em; color: #11a1f5;"> from excel file </span></p>
Read data from csv in Pandas

In [41]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', 85) # set Jupeter note book to dispay 85 columns

<p style="padding:5px; border-bottom:3px solid orange; background:#3c5c75; margin:25px 0 -7px 0;"> 
   <span style="font-size:1.5em; color:#f7ca25;" >Step 1 -</span>
   <span style="font-size:1.5em; color:white;">Read CSV files  - </span> 
   <span style="font-size:1.2em; color:#17cffc;">"Employee Target"</span>
</p> 

In [42]:
df = pd.read_csv("Data/2023/I1_Hrm/Employees/empTarget.csv", index_col=False ,encoding='unicode_escape').fillna(0)

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 1a - Remove Duplicate Line and Deleted Record</p>

In [43]:
DataA = df.drop_duplicates(subset=['empCode'], keep='last')
DataB = DataA.drop(DataA.loc[DataA['comment']=='Delete'].index)

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 1b - Combine Employee Code and Transaction Time</p>

In [44]:
DataB['combined'] = DataB['empCode'].astype(str)+" "+DataB['traDate']

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 1c- Select the Combine Column and convert it into a "LIST"</p>

In [45]:
List = DataB["combined"] #.tolist()

<p style="padding:5px; border-bottom:3px solid orange; background:#3c5c75; margin:25px 0 -7px 0;"> 
   <span style="font-size:1.5em; color:#f7ca25;" >Step 2 -</span>
   <span style="font-size:1.5em; color:white;">Read CSV files  - </span> 
   <span style="font-size:1.2em; color:#17cffc;">" Employee Target Detail"</span>
</p> 

In [46]:
df2 = pd.read_csv("Data/2023/I1_Hrm/Employees/empTargetDetail.csv", index_col=False ,encoding='unicode_escape').fillna(0)
### Combine Employee Code and Transaction Time.................
df2['combined'] = df2['empCode'].astype(str)+" "+df2['traDate']

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 3a- Filtere the 'Employee Target Detail' by Employee "Target List"</p>

In [47]:
filtered_df = df2[df2['combined'].isin(List)]

In [48]:
### Convert 'salesTarget' astype to "FLOAT" to avoid error if any numaric value contain float type.
filtered_df = filtered_df.astype({'salesTarget':'float'})

<p style="padding:5px; border-bottom:3px solid orange; background:#3c5c75; margin:25px 0 -7px 0;"> 
   <span style="font-size:1.5em; color:#f7ca25;" >Step 3 -</span>
   <span style="font-size:1.5em; color:white;">Create Pivot Table - </span> 
   <span style="font-size:1.2em; color:#17cffc;"> by Employee </span>
</p> 

In [56]:
empPT = pd.pivot_table(filtered_df,index=["empName","itemDesc"],values=["salesTarget","qty"],aggfunc=np.sum)
empPT = empPT.reset_index()

In [57]:
empGBy = empPT.groupby(['empName']).agg({'salesTarget': 'sum', 'qty': 'sum'}) 
empGBy.loc['z. Grand Total'] = empGBy.sum()
empGBy.reset_index(inplace=True)

In [58]:
empConcat = pd.concat([empPT, empGBy])
empSort = empConcat.sort_values(['empName', 'itemDesc'], ascending = [True, True], na_position ='last')
empSort['itemDesc'] = empSort['itemDesc'].fillna('Total')

<p style="font-family:Calibri; font-size:1.2em; color:#061233; padding:5px 5px; margin: 10px 0 -7px 20px;
border-bottom: 3px solid #0d1e4f; background: #e3e9fa">  
Step 3a- Employee Traget Percentager by Item</p>

In [39]:
### Convert into Dictionary--------------------
empGBy = empGBy[['empName','salesTarget']]
t1List = empGBy.values.tolist() ##Convert into List
mapDic1 = {code: name for code, name in t1List}
empSort['Total'] = empSort['empName'].map(mapDic1)
empSort['Percentage'] = (empSort['salesTarget'] / empSort['Total'])*100

In [54]:
### to Hide and Show----------------------------
empSort['Department'] = empSort['empName']
empSort['Vertical'] = empSort['empName']
empSort.loc[empSort["itemDesc"] != "Total", ["Vertical"]] = '---' #  rlSort["status"]

In [28]:
empData=[]
for i in range(empSort.shape[0]):
        temp=empSort.iloc[i]
        empData.append(dict(temp))